#Análisis de tweets contradictorios

Los tweets que se analizan han sido escritos por los miembros del grupo, al igual que la evaluación de si un tweet es tóxico o no

In [1]:
import pandas as pd
import numpy as np

In [2]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  COLLAB = True
else:
  COLLAB = False

Mounted at /content/drive


In [3]:
if COLLAB == True:
  df = pd.read_csv('/content/drive/Shareddrives/ETICA/tweets_contradictorios.csv')
else:
  df = pd.read_csv('../data/tweets_contradictorios.csv')

In [4]:
df

,tweet,manual_toxicity
0,Crees que Irene Montero está MALTRATANDO INJUS...,1
1,Descubro accidentalmente que Javier Ortega Smi...,1
2,Pedro Sanchez es genial hijos de puta,1
3,"Abascal, Ortega Smith, Espinosa, Monssterio y ...",1
4,"""Nazi en el Mediterráneo"" con Ortega Smith me ...",1
5,Por qué a la derecha no le gusta el Papa y por...,1
6,"Muy necesaria las vacunas a los niños, ya que ...",1
7,"Muy bien, Pedro. Un día menos para que te hech...",1
8,Lo de lograr la inmunidad de rebaño lo van con...,1
9,"El acoso que sufrió @PabloIglesias, @IreneMont...",1


In [5]:
!pip install transformers
import transformers
from transformers import pipeline
pipe = pipeline("text-classification", model="Newtral/xlm-r-finetuned-toxic-political-tweets-es")

     |████████████████████████████████| 3.4 MB 8.5 MB/s 
     |████████████████████████████████| 895 kB 43.3 MB/s 
     |████████████████████████████████| 61 kB 612 kB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/753 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/421 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [6]:
toxic_predicted = []
verytoxic_predicted = []
for idx, row in df.iterrows():
  row_tox_value = pipe(row.tweet, return_all_scores=True)[0][0]['score']
  row_vtox_value = pipe(row.tweet, return_all_scores=True)[0][1]['score']
  toxic_predicted.append(row_tox_value)
  verytoxic_predicted.append(row_vtox_value)
df['predicted_toxic'] = toxic_predicted
df['predicted_verytoxic'] = verytoxic_predicted

Consideramos que una toxicidad "toxic" de 50% ya cuenta como tóxico (valor 1) y a partir del 90% es bastante toxicidad y lo evaluamos con un 2. 
Hacemos lo mismo para "very toxic".

In [8]:
toxic_value_predicted = []
very_toxic_value_predicted = []
for idx, row in df.iterrows():
  if (row.predicted_toxic >= 0.9):
    toxic_value_predicted.append("2")
  elif (row.predicted_toxic >= 0.5):
    toxic_value_predicted.append("1")
  else:
    toxic_value_predicted.append("0")
  if (row.predicted_verytoxic >= 0.9):
    very_toxic_value_predicted.append("2")
  elif (row.predicted_verytoxic >= 0.5):
    very_toxic_value_predicted.append("1")
  else:
    very_toxic_value_predicted.append("0")
df['predicted_toxic_value'] = toxic_value_predicted
df['predicted_verytoxic_value'] = very_toxic_value_predicted

##Verdadero positivo
Tweets tóxicos predichos como tóxicos

In [17]:
vp = df.loc[(df['manual_toxicity'] == 1) & (df['predicted_toxic_value'] >= "1")]
vp

,tweet,manual_toxicity,predicted_toxic,predicted_verytoxic,predicted_toxic_value,predicted_verytoxic_value
0,Crees que Irene Montero está MALTRATANDO INJUS...,1,0.996480,0.947789,2,2
1,Descubro accidentalmente que Javier Ortega Smi...,1,0.994227,0.995985,2,2
2,Pedro Sanchez es genial hijos de puta,1,0.762539,0.270002,1,0
3,"Abascal, Ortega Smith, Espinosa, Monssterio y ...",1,0.991106,0.997693,2,2
5,Por qué a la derecha no le gusta el Papa y por...,1,0.965785,0.012007,2,0
7,"Muy bien, Pedro. Un día menos para que te hech...",1,0.990515,0.471767,2,0
8,Lo de lograr la inmunidad de rebaño lo van con...,1,0.988665,0.057677,2,0
12,"A Isabel Díaz Ayuso, después de esto, no le qu...",1,0.944200,0.010912,2,0
14,"Si tan lista es Ayuso, ¿por qué no deduce con ...",1,0.818915,0.003035,1,0
16,"Buen discurso de Ayuso, pero mis dos Cacatuas ...",1,0.638808,0.002658,1,0


In [18]:
vp.shape[0]

24

##Falso positivo
Tweets no tóxicos predichos como tóxicos

In [19]:
fp = df.loc[(df['manual_toxicity'] == 0) & (df['predicted_toxic_value'] >= "1")]
fp

,tweet,manual_toxicity,predicted_toxic,predicted_verytoxic,predicted_toxic_value,predicted_verytoxic_value
19,Monasterio pide a Ayuso quitar las subvencione...,0,0.995146,0.507088,2,1
38,"Lo que nosotros hemos hecho, cosa que no hizo ...",0,0.985027,0.113780,2,0
39,cuanto peor mejor para todos y cuanto peor par...,0,0.956062,0.012861,2,0
42,dejad de insultar,0,0.974321,0.256054,2,0
48,no se que son los menas,0,0.745069,0.067947,1,0
50,Cuando pensábamos que España se iba al carajo ...,0,0.996555,0.898260,2,1
54,"No sé por qué llamáis traidor a Pedro Sánchez,...",0,0.993559,0.859539,2,1
58,"Antes odiaba el PSOE, pero ahora no, me he vue...",0,0.561520,0.003900,1,0


In [20]:
fp.shape[0]

8

##Verdadero negativo
Tweets no tóxicos predichos como no tóxicos

In [21]:
vn = df.loc[(df['manual_toxicity'] == 0) & (df['predicted_toxic_value'] == "0")]
vn

,tweet,manual_toxicity,predicted_toxic,predicted_verytoxic,predicted_toxic_value,predicted_verytoxic_value
13,Me da pena lo que está ocurriendo con Ciudadanos,0,0.010621,0.001392,0,0
24,Dice Rocío Monasterio que jamás había conocido...,0,0.023922,0.001135,0,0
36,Gracias por tu gestión presi,0,0.007013,0.004124,0,0
37,"estoy de acuerdo, no estoy de acuerdo",0,0.037815,0.004860,0,0
40,"No he dormido nada, no me pregunten demasiado",0,0.149286,0.005949,0,0
41,Los españoles son muy españoles y mucho españoles,0,0.288580,0.005081,0,0
43,no voto al PSOE pero estoy de acuerdo,0,0.090415,0.002043,0,0
44,no voto al PP pero estoy de acuerdo,0,0.103786,0.001662,0,0
45,no voto a VOX pero estoy de acuerdo,0,0.179481,0.002989,0,0
46,no voto a Podemos pero estoy de acuerdo,0,0.147781,0.001850,0,0


In [22]:
vn.shape[0]

13

##Falso negativo
Tweets tóxicos predichos como no tóxicos

In [23]:
fn = df.loc[(df['manual_toxicity'] == 1) & (df['predicted_toxic_value'] == "0")]
fn

,tweet,manual_toxicity,predicted_toxic,predicted_verytoxic,predicted_toxic_value,predicted_verytoxic_value
4,"""Nazi en el Mediterráneo"" con Ortega Smith me ...",1,0.011205,0.001837,0,0
6,"Muy necesaria las vacunas a los niños, ya que ...",1,0.086665,0.000977,0,0
9,"El acoso que sufrió @PabloIglesias, @IreneMont...",1,0.015508,0.001632,0,0
10,Soy yo el unico que piensa que Irene Montero n...,1,0.105190,0.001427,0,0
11,"Buena propuesta de ley, no hagáis más.",1,0.010828,0.001811,0,0
15,La definición de tener más cara que espalda. S...,1,0.198267,0.001749,0,0
18,"Algunos por lo menos pagan el chalé, no como E...",1,0.372021,0.004637,0,0
23,"Con odio viven Santiago Abascal, Javier Ortega...",1,0.028546,0.001566,0,0
29,"@marianorajoy gracias por todo, chorizo",1,0.009470,0.004603,0,0
33,¿En qué universidad cursó usted la carrera? ¿E...,1,0.057087,0.001519,0,0


In [24]:
fn.shape[0]

14